In [1]:
!nvcc --version
%env OMP_NUM_THREADS=3

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
env: OMP_NUM_THREADS=3


In [2]:
%%writefile dfs.cpp


#include <iostream>
#include <vector>
#include <stack>
#include <omp.h>

using namespace std;

const int MAX = 100; // Maximum size for the graph

vector<int> graph[MAX]; // Graph representation
bool visited[MAX];      // Array to mark visited nodes

// Depth-First Search function
void dfs(int start_node) {
   stack<int> s;
   s.push(start_node);

   while (!s.empty()) {
       int current_node = s.top();
       s.pop();
       if (!visited[current_node]) {
           visited[current_node] = true;
           cout << current_node << " ";

           #pragma omp parallel for
           for (int i = 0; i < graph[current_node].size(); i++) {
               int adj_node = graph[current_node][i];
               if (!visited[adj_node]) {
                   #pragma omp critical
                   {
                       s.push(adj_node);
                   }
               }
           }
       }
   }
}

int main() {
   int n, m; // n: number of nodes, m: number of edges
   cout << "Enter the number of nodes and edges: ";
   cin >> n >> m;

   // Input edges
   cout << "Enter the edges (node pairs):\n";
   for (int i = 0; i < m; i++) {
       int u, v;
       cin >> u >> v;
       graph[u].push_back(v);
       graph[v].push_back(u); // For undirected graph
   }

   // Initialize visited array
   #pragma omp parallel for
   for (int i = 0; i < n; i++) {
       visited[i] = false;
   }

   cout << "Depth-First Search (DFS): ";
   dfs(0); // Start DFS from node 0
   cout << endl;

   return 0;
}


Writing dfs.cpp


In [3]:
!g++ dfs.cpp -o dfs -fopenmp

In [4]:
!./dfs

Enter the number of nodes and edges: 5 6
Enter the edges (node pairs):
0 1
0 2
1 3
1 4
2 4
3 4
Depth-First Search (DFS): 0 2 4 1 3 
